<a href="https://colab.research.google.com/github/Waye/CSC420-CourseWork-fall2019/blob/master/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')


#'/content/drive/My Drive/data/cat_data/'


Q2.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        # Number of parameters:
        # (5 * 5 * 1 + 2) * 16 = 324
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=5, padding=2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Number of parameters
        # (5 * 5 * 16 + 2) * 32 = 12832
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Number of parameters
        # (3 * 3 * 32 + 1) * 64) = 18496
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Number of parameters:
        # (3 * 3 * 64 + 1) * 128 = 73856
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Number of parameters
        # 12 * 12 * 128 * 3 = 55296
        self.fc = nn.Linear(12*12*64, 1)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [0]:
import gc

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from shapely.geometry.point import Point
from skimage.color import gray2rgb
from skimage.draw import circle, circle_perimeter_aa
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

#from nn_q2 import Net

use_gpu = torch.cuda.is_available()

class CircleDataset(Dataset):

    def __init__(self, size):
        self.size = size
        self.seeds = [np.random.randint(1, 2147483647) for i in range(size)]
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        seed = self.seeds[index]
        np.random.seed(seed)
        param, img = noisy_circle(200, 50, 2)
        np.random.seed(None)
        return img, param


def draw_circle(img, row, col, rad):
    rr, cc, val = circle_perimeter_aa(row, col, rad)
    valid = (
        (rr >= 0) &
        (rr < img.shape[0]) &
        (cc >= 0) &
        (cc < img.shape[1])
    )
    img[rr[valid], cc[valid]] = val[valid]


def noisy_circle(size, radius, noise):
    img = np.zeros((size, size), dtype=np.float)

    # Circle
    row = np.random.randint(size)
    col = np.random.randint(size)
    rad = np.random.randint(10, max(10, radius))
    draw_circle(img, row, col, rad)

    # Noise
    img += noise * np.random.rand(*img.shape)
    return (row, col, rad), img


def find_circle(img):
    # Load the CNNs
    netx = Net()
    nety = Net()
    netr = Net()
    device = torch.device('cuda')
    netx.load_state_dict(torch.load('q2x.pt'))
    nety.load_state_dict(torch.load('q2y.pt'))
    netr.load_state_dict(torch.load('q2r.pt'))
    netx.to(device)
    nety.to(device)
    netr.to(device)

    # Test the image
    img_tensor = img.reshape(200, 200, 1).transpose((2, 0, 1))
    tensor = torch.from_numpy(img_tensor)
    if use_gpu:
        tensor = tensor.cuda()
    tensor = tensor.unsqueeze(1).float()
    resx = netx(tensor).detach().cpu().numpy()[0]
    resy = nety(tensor).detach().cpu().numpy()[0]
    resr = netr(tensor).detach().cpu().numpy()[0]
    return int(resx), int(resy), int(resr)


def iou(params0, params1):
    row0, col0, rad0 = params0
    row1, col1, rad1 = params1

    shape0 = Point(row0, col0).buffer(rad0)
    shape1 = Point(row1, col1).buffer(rad1)

    return (
        shape0.intersection(shape1).area /
        shape0.union(shape1).area
    )


def observe():
    '''
    Plot the groundtruth bounding circle and the predicted bounding circle.
    The groundtruth is in green and the prediction in red.
    '''
    for _ in range(10):
        netx = Net()
        nety = Net()
        netr = Net()
        device = torch.device('cuda')
        netx.load_state_dict(torch.load('q2x.pt'))
        nety.load_state_dict(torch.load('q2y.pt'))
        netr.load_state_dict(torch.load('q2r.pt'))
        netx.to(device)
        nety.to(device)
        netr.to(device)

        params, img = noisy_circle(200, 50, 2)
        img_tensor = img.reshape(200, 200, 1).transpose((2, 0, 1))
        tensor = torch.from_numpy(img_tensor)
        tensor = tensor.unsqueeze(1).float()
        if use_gpu:
            tensor = tensor.cuda()
        resx = netx(tensor).detach().cpu().numpy()[0]
        resy = nety(tensor).detach().cpu().numpy()[0]
        resr = netr(tensor).detach().cpu().numpy()[0]
        img = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)

        rr, cc, val = circle_perimeter_aa(int(resx), int(resy), int(resr))
        valid = (
            (rr >= 0) &
            (rr < img.shape[0]) &
            (cc >= 0) &
            (cc < img.shape[1])
        )

        img[rr[valid], cc[valid], 0] = 255
        img[rr[valid], cc[valid], 1] = 0
        img[rr[valid], cc[valid], 2] = 0

        rr, cc, val = circle_perimeter_aa(params[0], params[1], params[2])
        valid = (
            (rr >= 0) &
            (rr < img.shape[0]) &
            (cc >= 0) &
            (cc < img.shape[1])
        )

        img[rr[valid], cc[valid], 0] = 0
        img[rr[valid], cc[valid], 1] = 255
        img[rr[valid], cc[valid], 2] = 0

        print(iou((int(resx), int(resy), int(resr)), params))
        plt.imshow(img)
        plt.show()


def iou_loss(predict, target):
    predict = predict.detach().cpu().numpy()
    target = target.detach().cpu().numpy()
    predict_arr = np.zeros((predict.shape[0], 1, 200, 200), dtype=np.float)
    target_arr = np.zeros((predict.shape[0], 1, 200, 200), dtype=np.float)
    for i in range(predict.shape[0]):
        p = predict[i]
        t = target[i]
        rr1, cc1 = circle(p[0], p[1], p[2])
        rr2, cc2 = circle(t[0], t[1], t[2])
        valid1 = (
            (rr1 >= 0) &
            (rr1 < 200) &
            (cc1 >= 0) &
            (cc1 < 200)
        )
        valid2 = (
            (rr2 >= 0) &
            (rr2 < 200) &
            (cc2 >= 0) &
            (cc2 < 200)
        )
        circle1 = np.zeros((200, 200), dtype=np.float)
        circle2 = np.zeros((200, 200), dtype=np.float)
        circle1[rr1[valid1], cc1[valid1]] = 1
        circle2[rr2[valid2], cc2[valid2]] = 1
        predict_arr[i,0,:,:] = circle1
        target_arr[i,0,:,:] = circle2
    predict = torch.tensor(predict_arr, requires_grad=True)
    target = torch.tensor(target_arr, requires_grad=True)

    intersect = predict * target
    intersect = intersect.view(predict.shape[0], 1, -1).sum(2)
    union = predict + target - (predict * target)
    union = union.view(predict.shape[0], 1, -1).sum(2)

    loss = intersect / union

    return 1 - loss.mean()


def train_model(models, trainloader, testloader, epochs, criterion, optimizer):
    mx, my, mr = models[0], models[1], models[2]
    ox, oy, Or = optimizer[0], optimizer[1], optimizer[2]
    for e in range(epochs):
        running_loss = 0
        for images, params in tqdm(trainloader):
            mx.train()
            my.train()
            mr.train()
            params = torch.stack(params, 1).float()
            if use_gpu:
                images = images.cuda()
                params = params.cuda()

            resx = mx(images.unsqueeze(1).float())
            resy = my(images.unsqueeze(1).float())
            resr = mr(images.unsqueeze(1).float())

            
            # loss = criterion(torch.stack((resx, resy, resr), dim=1), params) For IOU loss

            lossx = criterion(resx[:,0], params[:,0])
            lossy = criterion(resy[:,0], params[:,1])
            lossr = criterion(resr[:,0], params[:,2])
            ox.zero_grad()
            oy.zero_grad()
            Or.zero_grad()
            lossx.backward()
            lossy.backward()
            lossr.backward()
            
            # loss.backward()  # For IOU loss
            ox.step()
            oy.step()
            Or.step()
            running_loss += lossx.item() + lossy.item() + lossr.item()
            gc.collect()
        else:
            print(f"Epoch {e+1}")
            print(f"Training loss: {running_loss / len(trainloader)}")
            eval_model((mx, my, mr), testloader)
            # torch.save(mx.state_dict(), f'q2_checkpoints/q2_checkpoints_large2mx_{e+1}.pt')
            # torch.save(my.state_dict(), f'q2_checkpoints/q2_checkpoints_large2my_{e+1}.pt')
            # torch.save(mr.state_dict(), f'q2_checkpoints/q2_checkpoints_large2mr_{e+1}.pt')


def eval_model(models, testloader):
    mx, my, mr = models[0], models[1], models[2]
    mx.eval()
    my.eval()
    mr.eval()
    metric = 0
    for images, params in testloader:
        params = torch.stack(params, 1).float()
        if use_gpu:
            images = images.cuda()

        resx = mx(images.unsqueeze(1).float()).detach().cpu().numpy()[0]
        resy = my(images.unsqueeze(1).float()).detach().cpu().numpy()[0]
        resr = mr(images.unsqueeze(1).float()).detach().cpu().numpy()[0]
        res = (resx, resy, resr)
        params = params.numpy()[0]
        metric += iou(res, params)
    print(f"Average IoU for test set {metric / len(testloader)}")


def train():
    trainset = CircleDataset(200000)
    trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
    testset = CircleDataset(1000)
    testloader = DataLoader(testset, batch_size=1, shuffle=True)

    netx = Net()
    nety = Net()
    netr = Net()
    if use_gpu:
        netx = netx.cuda()
        nety = nety.cuda()
        netr = netr.cuda()

    criterion = nn.MSELoss()
    # criterion = iou_loss
    optimizerx = optim.Adam(netx.parameters(), lr=0.01)
    optimizery = optim.Adam(nety.parameters(), lr=0.01)
    optimizerr = optim.Adam(netr.parameters(), lr=0.01)
    epochs = 20
    train_model((netx, nety, netr), trainloader, testloader, epochs, criterion, (optimizerx, optimizery, optimizerr))
    eval_model((netx,nety,netr), testloader)


# test    
train()
exit(0)
results = []
for _ in range(1000):
    params, img = noisy_circle(200, 50, 2)
    detected = find_circle(img)
    results.append(iou(params, detected))
results = np.array(results)
print(results.mean())
print((results > 0.7).mean())

In [0]:
import argparse
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

#from nn import Net
from skimage import io, feature, transform


use_gpu = torch.cuda.is_available()


def binary_mask(arr):
    arr[arr > 0.5] = 1
    arr[arr <= 0.5] = 0


def process_cat(cat):
    h, w = cat.shape[:2]
    output_size = (128, 128)
    if isinstance(output_size, int):
        if h > w:
            new_h, new_w = output_size * h / w, output_size
        else:
            new_h, new_w = output_size, output_size * w / h
    else:
        new_h, new_w = output_size

    new_h, new_w = int(new_h), int(new_w)

    sized_cat = transform.resize(cat, (new_h, new_w))
    sized_cat = sized_cat.transpose((2, 0, 1))
    cat_tensor = torch.from_numpy(sized_cat)
    return cat_tensor


def plot_seg(image, net):
    cat = io.imread(image)
    cat_tensor = process_cat(cat)
    
    if use_gpu:
        cat_tensor = cat_tensor.cuda()
    cat_mask = net(cat_tensor.unsqueeze(0).float())
    img = cat_mask.detach().cpu().numpy()[0,0,:,:]
    resized_mask = transform.resize(img, cat.shape[0:2], anti_aliasing=False, preserve_range=True)
    binary_mask(resized_mask)
    resized_mask = (resized_mask * 255).astype(np.uint8)
    ret, thresh = cv2.threshold(resized_mask, 127,255, 0)
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(cat, contours, -1, (0,255,0), 3)
    return cat


def plot_mask(image, net):
    cat = io.imread(image[0])
    cat_tensor = process_cat(cat)
    truth = io.imread(image[1])

    if use_gpu:
        cat_tensor = cat_tensor.cuda()
    cat_mask = net(cat_tensor.unsqueeze(0).float())
    mask = cat_mask.detach().cpu().numpy()[0,0,:,:]
    resized_mask = transform.resize(mask, cat.shape[0:2], anti_aliasing=False, preserve_range=True)
    binary_mask(resized_mask)
    return cat, truth, resized_mask


def main(images, mask=False):
    net = Net()
    if use_gpu:
        device = torch.device('cuda')
        net.load_state_dict(torch.load('q1_checkpoints/q1_2_dice_20.pt'))
        net.to(device)
    else:
        net.load_state_dict(torch.load('q1_checkpoints/q1_2_bce_35.pt', map_location='cpu'))
    
    if mask:
        fig = plt.figure(figsize=(16, 20))
        l = len(images) // 2 + 1
        for i in range(len(images) // 2):
            cat, truth, pred = plot_mask(images[2 * i], net)
            a = fig.add_subplot(l, 6, i*6 + 1)
            imgplot = plt.imshow(cat)
            if i == 0: a.set_title('Cat')
            a = fig.add_subplot(l, 6, i*6 + 2)
            imgplot = plt.imshow(truth, cmap='gray')
            if i == 0: a.set_title('Groundtruth')
            a = fig.add_subplot(l, 6, i*6 + 3)
            imgplot = plt.imshow(pred, cmap='gray')
            if i == 0: a.set_title('Prediction')
            cat, truth, pred = plot_mask(images[2 * i + 1], net)
            a = fig.add_subplot(l, 6, i*6 + 4)
            imgplot = plt.imshow(cat)
            if i == 0: a.set_title('Cat')
            a = fig.add_subplot(l, 6, i*6 + 5)
            imgplot = plt.imshow(truth, cmap='gray')
            if i == 0: a.set_title('Groundtruth')
            a = fig.add_subplot(l, 6, i*6 + 6)
            imgplot = plt.imshow(pred, cmap='gray')
            if i == 0: a.set_title('Prediction')
        if len(images) % 2 != 0:
            cat, truth, pred = plot_mask(images[-1], net)
            a = fig.add_subplot(l, 6, i*6 + 7)
            imgplot = plt.imshow(cat)
            a = fig.add_subplot(l, 6, i*6 + 8)
            imgplot = plt.imshow(truth, cmap='gray')
            a = fig.add_subplot(l, 6, i*6 + 9)
            imgplot = plt.imshow(pred, cmap='gray')
        plt.savefig('vis1_2_dice', bbox_inches='tight')
    else:
        i = 1
        for image in images:
            cat = plot_seg(image[0], net)
            plt.imsave(f'seg{i}.png', cat)
            i += 1


# test visualize
parser = argparse.ArgumentParser(description='Visualize cat segmentations')
parser.add_argument('root', type=str, help='path to where files are located')
parser.add_argument('pickle', type=str, help='pickle file containing list of filenames')
parser.add_argument('--mask', action="store_true", help='display mask instead of outline')

args = parser.parse_args()
f = open(args.pickle, 'rb')
files = pickle.load(f)
new_files = []
for file in files:
  new_files.append((args.root + 'input/' + file[0], args.root + 'mask/' + file[1]))
# main(new_files, args.mask)